# Pydantic validation framework for the EMO BON observatory and other metadata log sheets

- **pydantic** Data validation using Python type hints.
    - [pypi](https://pypi.org/project/pydantic/)
    - [Documentation](https://docs.pydantic.dev/latest/)

In [1]:
import os
import sys

# Weird stuff from JupyterHub after I moved modules and notebooks around:
# For some reasong CWD is /src/scratch even though this notebook is in /srv/scratch/emo-bon-validation
# The terminal also show us to be in /srv/scratch/emo-bon-validation
# So...
if os.getcwd() == "/srv/scratch":
    os.chdir("./emo-bon-data-validation")
print(f"CWD is {os.getcwd()}")

import importlib
import subprocess
from pathlib import Path
import pandas as pd
import pydantic

CWD is /srv/scratch/emo-bon-data-validation


##### Init the directory structure

In [14]:
if False:
    #Init dirs and paths, write csv files
    # Init the validation classes dir if needed
    # Note that __init__.py will need be edited manually to import the validators
    # e.g from .observatories import Model as observatoriesModel
    validation_classes_path = "./validation_classes"
    if True:
        if not os.path.exists(validation_classes_path):
            os.mkdir(validation_classes_path)
            Path(os.path.join(validation_classes_path, "__init__.py")).touch()
            os.mkdir(raw_files_path)

## Governance data

#### Read each of the governance CSV files into a Pandas dataframe

In [15]:
github_path = "https://raw.githubusercontent.com/emo-bon/governance-data/main/"
file_names = [
        "logsheets.csv",              # contain the URLs of the googlesheets that are the logsheets
        "observatories.csv"           # contain information about each observatory
        #"organisations.csv",         # contain information about the organisations in EMO BON
        #"planned_events.csv"         # contains information about planned EMO BON events (this file is only used by humans, not by any actions) - DONT CARE
        #"ro-crate-metadata.json"     # IGNORE
        ]
dfs = {}
for f in file_names:
    df = pd.read_csv(os.path.join(github_path, f))
    print(f"This is info() for {df.info()}")
    dfs[f] = df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   EMBRC Node                           18 non-null     object
 1   EMBRC Site                           18 non-null     object
 2   EMOBON_observatory_id                18 non-null     object
 3   Water Column                         17 non-null     object
 4   Soft sediment                        7 non-null      object
 5   data_quality_control_threshold_date  18 non-null     object
 6   data_quality_control_assignee        18 non-null     object
 7   rocrate_profile_uri                  18 non-null     object
 8   autogenerate                         18 non-null     int64 
dtypes: int64(1), object(8)
memory usage: 1.4+ KB
This is info() for None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 22 columns):
 #   C

#### Validate Governance tables

In [16]:
from validation_classes import observatoriesModel, logsheetsModel
validator_class_paths = {"logsheets.csv": logsheetsModel, "observatories.csv": observatoriesModel}
validation_classes_path = "./validation_classes"

##### Observatories table

The observatories validator mostly changes the column names to make them consistent (and spelled correctly), removes blank strings ("   ") from cells, and reformats the dates.

In [17]:
file_name = "observatories.csv"
data = dfs[file_name] # dfs is dict of pandas df's
validator = validator_class_paths[file_name]
data_records = data.to_dict(orient="records")
validated_rows = [validator(**row).model_dump() for row in data_records]

#for row in data_records:
#    #print(row)
#    validator(**row)

#for record in validated_rows:
#    for field in record:
#        print(f"Record {field} has value {record[field]} is type {type(record[field])}")

ndf = pd.DataFrame.from_records(validated_rows, index="observatory_id")
ndf.to_csv(os.path.join("governance", "observatories_validated.csv"))

# Take a look at ./validation_classes/validated_governance/observatories_validated.csv

##### Logsheets table

In [ ]:
file_name = "logsheets.csv"
data = dfs[file_name] # dfs is dict of pandas df's
validator = validator_class_paths[file_name]
data_records = data.to_dict(orient="records")
validated_rows = [validator(**row).model_dump() for row in data_records]
        
ndf = pd.DataFrame.from_records(validated_rows, index="observatory_id")
ndf.to_csv(os.path.join(validation_classes_path, "governance", "logsheets_validated.csv"))

# Take a look at ./validation_classes/validated_governance/logsheets_validated.csv



## Logsheets from water column and soft sediments sampling and "measured" tables



In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
    
import os
import sys
import math
import pandas as pd
import pydantic
from validation_classes import samplingModel, measuredModel

def parse_sample_sheets(sampling_strategy: str, sheet_type: str, addresses: pd.core.frame.DataFrame) -> None: 
    for observatory in addresses:
        observatory_id, sheet_link = observatory
        #print(f"Observatory_id {observatory_id} sheet_link {sheet_link}")
        if not isinstance(sheet_link, str):
            #print(f"This is the sheet_link type {type(sheet_link)}")
            if isinstance(sheet_link, float): 
                if math.isnan(sheet_link):
                    print(f"Observatory {observatory_id} lacks valid sheet URL {sheet_link}")
                    continue
            else:
                raise ValueError(f"Unknown link {sheet_link} to observatory {observatory_id}")
        else:

            if observatory_id == "Plenzia": continue # Sheets not publically available
            # UMF soft_sed has two source_mat_ids
            if sampling_strategy == "soft_sediment" and observatory_id == "UMF":
                continue
            
            #if observatory_id in ["BPNS", "Bergen", "ESC68N", "MBAL4",
            #                     "VB", "ROSKOGO"]:
            #["Bergen", "MBAL4", "ESC68N", 
            #               "BPNS", "VB", "ROSKOGO",
            #               "EMT21", "PiEGetxo", "RFormosa",
            #              "OSD74", "AAOT", "NRMCB",
            #              "HCMR-1", "IUIEilat", "UMF"]:
            #    continue
    
            #if observatory_id != "BPNS":
            #    continue
    
            print(f"Processing {observatory_id}...")
            sampling_sheet_base = sheet_link.split("/edit")[0]
            sampling_sheet_suffix = "/gviz/tq?tqx=out:csv&sheet=%s"
            sample_sheet_link = sampling_sheet_base + sampling_sheet_suffix % sheet_type
            print(f"Sample sheet link: {sample_sheet_link}")
            df = pd.read_csv(sample_sheet_link, encoding='utf-8')
            data_records_all = df.to_dict(orient="records")
    
            # Many sheets have partially filled rows
            # The source_mat_id is manually curated and the PRIMARY_KEY
            # Therefore filter records on source_mat_id
            def filter_on_source_mat_id(d):
                # Bergen has it as source_material_id
                try:
                    value = d["source_mat_id"]
                except KeyError:
                    try:
                        value = d["source_material_id"]
                    except KeyError:
                        raise ValueError("Cannot find source_mat_id field")
                if isinstance(value, float):
                    if math.isnan(value):
                        return False
                elif value is None:
                    return False
                # Remove mis-formatted
                elif len(value.split("_")) < 6:
                    return False
                #Edge case of this otherwise blank entry having 6 "bits"
                elif value == "EMOBON_VB_Wa_230509_um_":
                    return False 
                else:
                    return True
                
            data_records_filtered = list(filter(filter_on_source_mat_id, data_records_all))
    
            if len(data_records_all) > len(data_records_filtered):
                print(f"Discarded {len(data_records_all) - len(data_records_filtered)} records leaving {len(data_records_filtered)}.")
            
            validator = validator_classes[sheet_type]
            validated_rows = [validator(**row).model_dump() for row in data_records_filtered]
    
            #for record in validated_rows:
            #    for field in record:
            #        print(f"Record {field} has value {record[field]} is type {type(record[field])}")
    
            save_dir = "./logsheets"
            outfile_name = f"{observatory_id}_{sampling_strategy}_{sheet_type}_validated.csv"
            ndf = pd.DataFrame.from_records(validated_rows, index="source_mat_id")
            ndf.to_csv(os.path.join(save_dir, outfile_name))
            print(f"Written {os.path.join(save_dir, outfile_name)}")


validator_classes = {"sampling": samplingModel, "measured": measuredModel}
# Get list of all URL links to sampling sheets
# NB  you cant use a "with" closure here when reading the Pandas df
governance_logsheets_validated_csv = "./governance/logsheets_validated.csv"
df = pd.read_csv(governance_logsheets_validated_csv)
water_column_sheet_addresses = df[["observatory_id", "water_column"]].values.tolist()
soft_sediment_sheet_addresses  = df[["observatory_id", "soft_sediment"]].values.tolist()
del df
    
parse_sample_sheets("water_column", "sampling", water_column_sheet_addresses)
parse_sample_sheets("soft_sediment", "sampling", soft_sediment_sheet_addresses)
parse_sample_sheets("water_column", "measured", water_column_sheet_addresses)
parse_sample_sheets("soft_sediment", "measured", soft_sediment_sheet_addresses)


# Refactoring

In [6]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
    
import os
import sys
import math
import pickle
import pandas as pd
from pydantic import ValidationError
from pprint import pprint
from validation_classes import samplingModel, measuredModel, samplingModelStrict

############################ CAUTION #############################################
STRICT      = True # As defined by Ioulia
SEMI_STRICT = True # As defined by Ioulia but not checking for Mandatory fields
##################################################################################

def parse_sample_sheets(sampling_strategy: str,
                        sheet_type: str,
                        addresses: pd.core.frame.DataFrame,
                       ) -> None:

    ################# temp ###################
    #addresses = [addresses[0]]
    #print(addresses)
    ################ temp ######################
    
    
    for observatory in addresses:
        observatory_id, sheet_link = observatory
        #print(f"Observatory_id {observatory_id} sheet_link {sheet_link}")
        if not isinstance(sheet_link, str):
            #print(f"This is the sheet_link type {type(sheet_link)}")
            if isinstance(sheet_link, float): 
                if math.isnan(sheet_link):
                    print(f"Observatory {observatory_id} lacks valid sheet URL {sheet_link}")
                    continue
            else:
                raise ValueError(f"Unknown link {sheet_link} to observatory {observatory_id}")
        else:

            if observatory_id == "Plenzia": continue # Sheets not publically available
            # UMF soft_sed has two source_mat_ids
            if sampling_strategy == "soft_sediment" and observatory_id == "UMF":
                continue
            
            #if observatory_id in ["BPNS", "Bergen", "ESC68N", "MBAL4",
            #                     "VB", "ROSKOGO"]:
            #["Bergen", "MBAL4", "ESC68N", 
            #               "BPNS", "VB", "ROSKOGO",
            #               "EMT21", "PiEGetxo", "RFormosa",
            #              "OSD74", "AAOT", "NRMCB",
            #              "HCMR-1", "IUIEilat", "UMF"]:
            #    continue
    
            #if observatory_id != "BPNS":
            #    continue
    
            print(f"Processing {observatory_id}...")
            sampling_sheet_base = sheet_link.split("/edit")[0]
            sampling_sheet_suffix = "/gviz/tq?tqx=out:csv&sheet=%s"
            sample_sheet_link = sampling_sheet_base + sampling_sheet_suffix % sheet_type
            print(f"Sample sheet link: {sample_sheet_link}")
            df = pd.read_csv(sample_sheet_link, encoding='utf-8')
            data_records_all = df.to_dict(orient="records")
    
            # Many sheets have partially filled rows
            # The source_mat_id is manually curated and the PRIMARY_KEY
            # Therefore filter records on source_mat_id
            def filter_on_source_mat_id(d):
                # Bergen has it as source_material_id
                try:
                    value = d["source_mat_id"]
                except KeyError:
                    try:
                        value = d["source_material_id"]
                    except KeyError:
                        raise ValueError("Cannot find source_mat_id field")
                if isinstance(value, float):
                    if math.isnan(value):
                        return False
                elif value is None:
                    return False
                # Remove mis-formatted
                elif len(value.split("_")) < 6:
                    return False
                #Edge case of this otherwise blank entry having 6 "bits"
                elif value == "EMOBON_VB_Wa_230509_um_":
                    return False 
                else:
                    return True
                
            data_records_filtered = list(filter(filter_on_source_mat_id, data_records_all))
    
            if len(data_records_all) > len(data_records_filtered):
                print(f"Discarded {len(data_records_all) - len(data_records_filtered)} records leaving {len(data_records_filtered)}.")

            if STRICT:
                model_type = f"{sheet_type}_strict"
            else:
                model_type = sheet_type

            validator = validator_classes[model_type]

            #validated_rows = [validator(**row).model_dump() for row in data_records_filtered]
            validated_rows = []
            errors: List[List[str:List[Dict]]] = [] # where each error is the inner Dict
            for row in data_records_filtered:
                try:
                    vr = validator(**row)
                except ValidationError as e:
                    if observatory_id == "Bergen":
                        errors.append([(row["source_material_id"], e.errors())])
                    else:
                        errors.append([(row["source_mat_id"], e.errors())])
                else:
                    validated_rows.append(vr)

            if errors:
                # errors is a list of lists where each inner list is a dict of row errors
                # where each isof key = source_mat_id and values is list of dicts each of which
                # is an error:
                #List[List[str:List[Dict]]]
                total_number_errors = sum([len(row[1]) for e in errors for row in e])
                print(f"Errors were found... {total_number_errors} in total")
                save_dir = "./validation_errors"
                outfile_name_pk = f"{observatory_id}_{sampling_strategy}_{model_type}_ERRORS.pickle"
                out_path_pk = os.path.join(save_dir, outfile_name_pk)
                with open(out_path_pk, "wb") as f:
                    pickle.dump(errors, f, pickle.HIGHEST_PROTOCOL)
                outfile_name_log = f"{observatory_id}_{sampling_strategy}_{model_type}_ERRORS.log"
                out_path_log = os.path.join(save_dir, outfile_name_log)                
                with open(out_path_log, "w") as f:
                    pprint(errors, f)

            else:
                assert len(validated_rows) == len(data_records_filtered), "Not sure what happenned, but len(validated_rows) != len(data_filtered_records)"
            
                #for record in validated_rows:
                #    for field in record:
                #        print(f"Record {field} has value {record[field]} is type {type(record[field])}")
    
                save_dir = "./logsheets"
                outfile_name = f"{observatory_id}_{sampling_strategy}_{model_type}_validated.csv"
                ndf = pd.DataFrame.from_records(validated_rows, index="source_mat_id")
                ndf.to_csv(os.path.join(save_dir, outfile_name))
                print(f"Written {os.path.join(save_dir, outfile_name)}")


validator_classes = {"sampling": samplingModel, "measured": measuredModel, "sampling_strict": samplingModelStrict}

# Get list of all URL links to sampling sheets
# NB  you cant use a "with" closure here when reading the Pandas df
governance_logsheets_validated_csv = "./governance/logsheets_validated.csv"
df = pd.read_csv(governance_logsheets_validated_csv)
water_column_sheet_addresses = df[["observatory_id", "water_column"]].values.tolist()
soft_sediment_sheet_addresses  = df[["observatory_id", "soft_sediment"]].values.tolist()
del df

parse_sample_sheets("water_column", "sampling", water_column_sheet_addresses)

#parse_sample_sheets("water_column", "sampling", water_column_sheet_addresses)
#parse_sample_sheets("soft_sediment", "sampling", soft_sediment_sheet_addresses)
#parse_sample_sheets("water_column", "measured", water_column_sheet_addresses)
#parse_sample_sheets("soft_sediment", "measured", soft_sediment_sheet_addresses)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processing ESC68N...
Sample sheet link: https://docs.google.com/spreadsheets/d/11_Eu0W1-sDiuzKx1cIl6YuxjRHmWezN6u9v3Ly8JZ3A/gviz/tq?tqx=out:csv&sheet=sampling
Discarded 114 records leaving 150.
Errors were found... 2319 in total
Processing Bergen...
Sample sheet link: https://docs.google.com/spreadsheets/d/1HuXHiUJICZrmCrJ4EZDyU5aSCMzDAc1cy_tne5YVPTg/gviz/tq?tqx=out:csv&sheet=sampling
Discarded 32 records leaving 108.
Errors were found... 1672 in total
Processing MBAL4...
Sample sheet link: https://docs.google.com/spreadsheets/d/1xfrqraPa0auQ1O-C9RUo68RhxrPCDWkVMCAUbj79AZI/gviz/tq?tqx=out:csv&sheet=sampling
Discarded 2 records leaving 78.
Errors were found... 935 in total
Processing BPNS...
Sample sheet link: https://docs.google.com/spreadsheets/d/1mEi4Bd2YR63WD0j54FQ6QkzcUw_As9Wilue9kaXO2DE/gviz/tq?tqx=out:csv&sheet=sampling
Discarded 40 records leaving 280.
Errors were found... 4635 in total
Proce